#Structured Streaming
## Overview
Demonstrating structured streaming with wikimedia.com/recentchanges data, where im "analysing" data in "real-time"
Its not real-time since im using 4 files to read data from, to simulate streaming.
As for analysing data, im doing simple count of types of changes, and showing them.

In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,LongType,BooleanType
from time import sleep
# Create a Spark session
spark = SparkSession.builder.appName("CSVStreamExample").getOrCreate()
#df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/mgajic9821rn@raf.rs/kafka_wikimedia.csv")
# Read CSV as a streaming DataFrame
csv_path = "dbfs:/FileStore/shared_uploads/mgajic9821rn@raf.rs/struct"
schema1 = StructType([
    StructField("$schema", StringType(), True),
    StructField("id", LongType(), True),
    StructField("type", StringType(), True),
    StructField("namespace", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("title_url", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("user", StringType(), True),
    StructField("bot", BooleanType(), True),
    StructField("notify_url", StringType(), True),
    StructField("minor", BooleanType(), True),
    StructField("length", IntegerType(), True),
    StructField("revision", StringType(), True),
    StructField("server_url", StringType(), True),
    StructField("server_name", StringType(), True),
    StructField("server_script_path", StringType(), True),
    StructField("wiki", StringType(), True),
    StructField("parsedcomment", StringType(), True),
    StructField("length_new", IntegerType(), True),
    StructField("revision_new", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("request_id", StringType(), True),
    StructField("dt", StringType(), True),
    StructField("domain", StringType(), True),
    StructField("stream", StringType(), True),
    StructField("topic", StringType(), True),
    StructField("partition", IntegerType(), True),
    StructField("offset", LongType(), True),
])
from pyspark.sql.functions import *
%matplotlib inline
streaming_df=spark.readStream.schema(schema1).option("maxFilesPerTrigger", 1).csv(csv_path)


#query.awaitTermination()


In [0]:
streamingCountsDF=(streaming_df.groupBy("type").count())
#display(streamingCountsDF)
print(streamingCountsDF.isStreaming)


query = (
    streamingCountsDF
    .writeStream
    .outputMode("complete")
    .format("memory")  # Use 'memory' to store in-memory table
    .queryName("counts")
   .trigger(processingTime="1 seconds")
    .start()
)


True


Sleeping until streaming actually starts.

In [0]:
while not query.recentProgress:
    sleep(1)

#Visualizing the data.
The count>100 is because there is some data that is wrong, and while it doesnt change anything, the graph looks better like this


In [0]:
%sql select * from counts where count>100 order by count desc;

type,count
edit,5525
categorize,4377
new,495


Databricks visualization. Run in Databricks to view.

giving streaming time to get more data


In [0]:
sleep(8)

In [0]:
%sql select * from counts where count>100 order by count desc;

type,count
edit,5525
categorize,4377
new,495


Databricks visualization. Run in Databricks to view.

In [0]:
sleep(5)

In [0]:
%sql select * from counts where count>100 order by count desc;

type,count
edit,12460
categorize,9229
new,922


Databricks visualization. Run in Databricks to view.

In [0]:
sleep(5)

In [0]:
%sql select * from counts where count>100 order by count desc;

type,count
edit,17226
categorize,12714
new,1131
log,105


Databricks visualization. Run in Databricks to view.

In [0]:
query.stop()